# Kickstarter one-hot Encoding to npz

### Extract the data from the csv

In [1]:
import numpy as np
from sklearn import preprocessing
from pickle import dump


In [17]:
raw_csv_data = np.loadtxt('ks-projects-201801-one-hot-v4.csv', delimiter = ',')
unscaled_inputs_all = raw_csv_data[:,1:]
targets_all = raw_csv_data[:,0]
print(unscaled_inputs_all.shape)


(365513, 20)


### Balance the dataset

In [6]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [7]:
scaler = preprocessing.StandardScaler().fit(unscaled_inputs_equal_priors)
scaler

StandardScaler()

In [8]:
scaler.mean_

array([3.67868902e+04, 3.21497626e+01, 9.10173438e-02, 1.22287663e-01,
       7.86694993e-01, 7.75596147e-02, 3.21549000e-02, 2.20842847e-02,
       1.17188091e-02, 8.02057974e-02, 5.67454977e-02, 1.68236735e-01,
       6.20756657e-02, 9.47749233e-02, 1.14012671e-02, 1.41219210e-01,
       2.76979723e-02, 1.02959188e-01, 7.79565421e-02, 3.32095928e-02])

In [9]:
scaler.scale_

array([1.01169771e+06, 1.14186753e+01, 2.87633772e-01, 3.27617750e-01,
       4.09641283e-01, 2.67477328e-01, 1.76411344e-01, 1.46957712e-01,
       1.07617278e-01, 2.71611538e-01, 2.31355670e-01, 3.74076377e-01,
       2.41292929e-01, 2.92903802e-01, 1.06166276e-01, 3.48247533e-01,
       1.64106047e-01, 3.03905567e-01, 2.68103188e-01, 1.79183469e-01])

In [10]:
scaler.transform(unscaled_inputs_equal_priors)

array([[-0.03484533,  2.26385607,  3.16020838, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.00670842,  2.3514319 , -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [ 0.00811815,  1.03779441, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       ...,
       [-0.03542253,  0.86264274, -0.31643483, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.03188411, -0.27584308, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.03371729, -0.27584308, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849]])

In [11]:
# Save the scaler
dump(scaler, open('scaler_kicsktarter_v4.pkl', 'wb'))

In [12]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [13]:
scaled_inputs

array([[-0.03484533,  2.26385607,  3.16020838, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.00670842,  2.3514319 , -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [ 0.00811815,  1.03779441, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       ...,
       [-0.03542253,  0.86264274, -0.31643483, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.03188411, -0.27584308, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.03371729, -0.27584308, -0.31643483, ..., -0.33878678,
        -0.29077066, -0.18533849]])

### Shuffle the data

In [14]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [15]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

105932.0 211625 0.5005646780862375
13199.0 26453 0.4989604203682002
13135.0 26454 0.49652226506388447


### Save the three datasets in *.npz

In [16]:
np.savez('kickstarter_data_train_v4', inputs=train_inputs, targets=train_targets)
np.savez('kickstarter_data_validation_v4', inputs=validation_inputs, targets=validation_targets)
np.savez('kickstarter_data_test_v4', inputs=test_inputs, targets=test_targets)